In [92]:
import pyspark.sql
from pyspark.sql import *
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate() # On crée la session

# On récupère le csv et on définit les paramétres - InferSchema permet de récupérer les infos comme le typage
data = spark.read.csv("../Data/demographics_de.csv", header=True, inferSchema=True) 

data.printSchema() # On affiche le schéma

data.createOrReplaceTempView("demo") # On crée une vue temporaire en la nommant

table = spark.sql("select * from demo") # On interroge la vu crée precédemment
table.show() # Affichage du DataFrame

root
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age_group: string (nullable = true)
 |-- population: integer (nullable = true)

+------------------+------+---------+----------+
|             state|gender|age_group|population|
+------------------+------+---------+----------+
|Baden-Wuerttemberg|female|    00-04|    261674|
|Baden-Wuerttemberg|female|    05-14|    490822|
|Baden-Wuerttemberg|female|    15-34|   1293488|
|Baden-Wuerttemberg|female|    35-59|   1919649|
|Baden-Wuerttemberg|female|    60-79|   1182736|
|Baden-Wuerttemberg|female|    80-99|    419471|
|Baden-Wuerttemberg|  male|    00-04|    274882|
|Baden-Wuerttemberg|  male|    05-14|    517387|
|Baden-Wuerttemberg|  male|    15-34|   1423367|
|Baden-Wuerttemberg|  male|    35-59|   1955828|
|Baden-Wuerttemberg|  male|    60-79|   1065669|
|Baden-Wuerttemberg|  male|    80-99|    264560|
|            Bayern|female|    00-04|    306378|
|            Bayern|female|    05-14|    560549|
| 

## Exercice 1: Afficher les cinq premières lignes du DataFrame.



In [93]:
Five_First = table.limit(5).show()

+------------------+------+---------+----------+
|             state|gender|age_group|population|
+------------------+------+---------+----------+
|Baden-Wuerttemberg|female|    00-04|    261674|
|Baden-Wuerttemberg|female|    05-14|    490822|
|Baden-Wuerttemberg|female|    15-34|   1293488|
|Baden-Wuerttemberg|female|    35-59|   1919649|
|Baden-Wuerttemberg|female|    60-79|   1182736|
+------------------+------+---------+----------+



## Exercice 2: Compter le nombre total de lignes dans le DataFrame.



In [94]:
Count_Lines = table.count()
print(Count_Lines)

192


## Exercice 3: Trouver le nombre distinct d'états (states) présents dans le DataFrame.



In [95]:
Nbr_Distinct_Etat = table.select(col('state')).distinct().count()
print(Nbr_Distinct_Etat)

16


## Exercice 4: Calculer la somme de la population totale.



In [96]:
Total_Population = table.select(col('population')).agg(sum(col('population')))

## Exercice 5: Afficher la population maximale par groupe d'âge (age_group).



In [97]:
Population_Max_Par_Age = table\
                            .select('age_group', 'population')\
                            .groupby(col('age_group'))\
                            .agg(max(col('population')))\
                            .orderBy(col('age_group'))\
                            .show()


+---------+---------------+
|age_group|max(population)|
+---------+---------------+
|    00-04|         440708|
|    05-14|         832251|
|    15-34|        2188068|
|    35-59|        3147565|
|    60-79|        2009976|
|    80-99|         729148|
+---------+---------------+



## Exercice 6: Filtrer les lignes du DataFrame pour n'inclure que les états (states) dont la population dépasse 1 million.



In [98]:
Etat_Sup_1Million = table.filter(col('population') > 1000000).show()

+-------------------+------+---------+----------+
|              state|gender|age_group|population|
+-------------------+------+---------+----------+
| Baden-Wuerttemberg|female|    15-34|   1293488|
| Baden-Wuerttemberg|female|    35-59|   1919649|
| Baden-Wuerttemberg|female|    60-79|   1182736|
| Baden-Wuerttemberg|  male|    15-34|   1423367|
| Baden-Wuerttemberg|  male|    35-59|   1955828|
| Baden-Wuerttemberg|  male|    60-79|   1065669|
|             Bayern|female|    15-34|   1507659|
|             Bayern|female|    35-59|   2310659|
|             Bayern|female|    60-79|   1421273|
|             Bayern|  male|    15-34|   1639500|
|             Bayern|  male|    35-59|   2348310|
|             Bayern|  male|    60-79|   1285577|
|             Hessen|female|    35-59|   1111053|
|             Hessen|  male|    35-59|   1111560|
|      Niedersachsen|female|    35-59|   1392647|
|      Niedersachsen|  male|    35-59|   1391992|
|Nordrhein-Westfalen|female|    15-34|   2047533|


## Exercice 7: Calculer la moyenne de la population pour chaque groupe d'âge (age_group).



In [99]:
Moyenne_Population_Par_Age = table.select('population', 'age_group').groupby(col('age_group')).agg(avg(col('population'))).show()

+---------+---------------+
|age_group|avg(population)|
+---------+---------------+
|    80-99|    168409.5625|
|    05-14|    230138.0625|
|    35-59|   910557.46875|
|    60-79|     562135.625|
|    00-04|   122699.90625|
|    15-34|   600409.78125|
+---------+---------------+



## Exercice 8: Trier le DataFrame par ordre décroissant de population.



In [100]:
Tri_Desc_Population = table.sort(col('population')).show()

+--------------------+------+---------+----------+
|               state|gender|age_group|population|
+--------------------+------+---------+----------+
|              Bremen|  male|    80-99|     15906|
|              Bremen|female|    00-04|     16202|
|              Bremen|  male|    00-04|     17380|
|            Saarland|female|    00-04|     19857|
|            Saarland|  male|    00-04|     21008|
|            Saarland|  male|    80-99|     26915|
|              Bremen|female|    80-99|     27937|
|              Bremen|female|    05-14|     28300|
|              Bremen|  male|    05-14|     30195|
|Mecklenburg-Vorpo...|female|    00-04|     33432|
|Mecklenburg-Vorpo...|  male|    00-04|     34712|
|             Hamburg|  male|    80-99|     38042|
|            Saarland|female|    05-14|     38101|
|            Saarland|  male|    05-14|     40909|
|Mecklenburg-Vorpo...|  male|    80-99|     42614|
|      Sachsen-Anhalt|female|    00-04|     44389|
|          Thueringen|female|  

## Exercice 9: Sélectionner les lignes du DataFrame où l'âge (age_group) est "18-24" et le sexe (gender) est "F".



In [101]:
Femme_Age1824 = table.filter((col('age_group') == '15-34') & (col('gender')=='female')).show()

+--------------------+------+---------+----------+
|               state|gender|age_group|population|
+--------------------+------+---------+----------+
|  Baden-Wuerttemberg|female|    15-34|   1293488|
|              Bayern|female|    15-34|   1507659|
|              Berlin|female|    15-34|    477479|
|         Brandenburg|female|    15-34|    213065|
|              Bremen|female|    15-34|     83990|
|             Hamburg|female|    15-34|    246744|
|              Hessen|female|    15-34|    715254|
|Mecklenburg-Vorpo...|female|    15-34|    146826|
|       Niedersachsen|female|    15-34|    875509|
| Nordrhein-Westfalen|female|    15-34|   2047533|
|     Rheinland-Pfalz|female|    15-34|    448341|
|            Saarland|female|    15-34|    103235|
|             Sachsen|female|    15-34|    386154|
|      Sachsen-Anhalt|female|    15-34|    193271|
|  Schleswig-Holstein|female|    15-34|    305125|
|          Thueringen|female|    15-34|    191712|
+--------------------+------+--

## Exercice 10: Ajouter une nouvelle colonne "population_percentage" qui représente la proportion de la population par rapport à la population totale.


In [102]:
add_column = table.withColumn('population_percentage', (col('population')*100/Total_Population.collect()[0][0]))
add_column.show()

+------------------+------+---------+----------+---------------------+
|             state|gender|age_group|population|population_percentage|
+------------------+------+---------+----------+---------------------+
|Baden-Wuerttemberg|female|    00-04|    261674|  0.31519691712808695|
|Baden-Wuerttemberg|female|    05-14|    490822|   0.5912149516522157|
|Baden-Wuerttemberg|female|    15-34|   1293488|   1.5580586146968172|
|Baden-Wuerttemberg|female|    35-59|   1919649|    2.312294866008908|
|Baden-Wuerttemberg|female|    60-79|   1182736|   1.4246533510261052|
|Baden-Wuerttemberg|female|    80-99|    419471|   0.5052697861638366|
|Baden-Wuerttemberg|  male|    00-04|    274882|   0.3311064873621483|
|Baden-Wuerttemberg|  male|    05-14|    517387|   0.6232135686470552|
|Baden-Wuerttemberg|  male|    15-34|   1423367|   1.7145031235119033|
|Baden-Wuerttemberg|  male|    35-59|   1955828|   2.3558739348685465|
|Baden-Wuerttemberg|  male|    60-79|   1065669|   1.2836414144277664|
|Baden